In [ ]:
# Getting started

In [1]:
push!(LOAD_PATH, "../../../ExoJulia/")

3-element Array{ByteString,1}:
 "/Applications/Julia-0.4.5.app/Contents/Resources/julia/local/share/julia/site/v0.4"
 "/Applications/Julia-0.4.5.app/Contents/Resources/julia/share/julia/site/v0.4"      
 "../../../ExoJulia/"                                                                

In [ ]:
Pkg.add("LsqFit")

In [2]:
using ExoJulia
using PyPlot
using LsqFit

//anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
include("utils.jl")
include("orbital_utils.jl")
include("rv.jl")

agol_periodogram (generic function with 1 method)

### Read-in Mystery RV data

In [4]:
numbers = readdlm("mystery_planet.txt");

In [5]:
time = numbers[:,1];
rv = numbers[:,2];
err = numbers[:,end];

In [ ]:
plot(time, rv, "o", ms=2.0)
xlabel("Time")
ylabel("RV")

## Period Fitting

### Agol Method

In [6]:
periods = linspace(1.0, 365.0, 1000)

linspace(1.0,365.0,1000)

In [7]:
p = collect(periods);

In [8]:
@time best_period = agol_periodogram(numbers, p)
# [time rv err]

116.50350350350351

0.804658 seconds (645.43 k allocations: 37.056 MB, 2.35% gc time)


## RV Fitting

In [ ]:
time_fold = numbers[:,1]; 
RV_fold = numbers[:,2];
err_fold = numbers[:,3];

In [ ]:
function func1(x, p::Vector)
    # p = [h,c,v0]
    p[1].*cos(x) .+ p[2].*sin(x) .+ p[3]
end

In [ ]:
function func2(x, p::Vector)
    # p = [K, w, ecc, gamma]
    #x ==  time, but we want dat f
    f = zeros(length(x))
    for i=1:length(x)
        M = mean_anomaly(best_period, x[i], 0.0)
        f[i] = f_from_M!(p[3], M)
    end 
    p[1] .* (cos(p[2] .+ f) .+ p[3] .* cos(p[2])) .+ p[4]
end

In [ ]:
@time fit2 = curve_fit(func2, time_fold, RV_fold, 1.0./err_fold.^2, [400.0, 0.0, 0.1, -1000.]);

In [ ]:
@time fit1 = curve_fit(func1, time_fold, RV_fold, 1.0./err_fold.^2, [100.0, 0.0, 0.0]);

In [ ]:
fit1.param

In [ ]:
fit_params = fit2.param

In [ ]:
estimate_errors(fit1)

In [ ]:
estimate_covar(fit2)

In [ ]:
numbers[:,1] = mod(numbers[:,1] - numbers[1,1], best_period);

In [ ]:
fastsortrows(numbers, [1]);

In [ ]:
numbers

In [ ]:
plot(numbers[:,1], numbers[:,2], "o", ms=2.5)
xlabel("Days")
ylabel("RV")

In [ ]:
K = fit_params[1];
w = fit_params[2];
ecc = fit_params[3]; 
gamma = fit_params[4];

In [ ]:
vrad_fit = zeros(length(time))
for i=1:length(time)
    M = mean_anaomoly!(best_period, time[i], time[1])
    f = f_from_M!(ecc, M)
    vrad_fit[i] = v_rad!(K, w, f, ecc, gamma)
    
end

In [ ]:
function fit_to_physical(h::Float64,c::Float64,v0::Float64)
    w = atan(-c/h)
    K = h / cos(w)
    
end

In [ ]:
plot(numbers[:,1], numbers[:,2], "o", ms=5)
plot(time_fold, vrad_fit, "o")
xlabel("Days")
ylabel("RV")

In [9]:
using Optim

### Trying an MCMC

In [ ]:
Pkg.add("Lora")

In [ ]:
using Lora

### Matrix Approach

In [10]:
function WH_W(err::Array{Float64,1})
    # Compose Wright & Howard W matrix
    W = zeros(Float64,length(err), length(err))
    for i=1:length(err)
        W[i,i] = 1./err[i]^2
    end 
    return W
end 

WH_W (generic function with 1 method)

In [11]:
@time W = WH_W(err);

  0.021063 seconds (20.31 k allocations: 906.292 KB)


In [12]:
function f_from_t(P::Float64, ecc::Float64, t::Float64, tp::Float64)
    f_from_M(ecc, mean_anomaly(P, t, tp))
end 

f_from_t (generic function with 1 method)

In [13]:
function WH_F(P::Float64, ecc::Float64, t::Array{Float64, 1}, tp::Float64; Nplanets::Int=1)
    # Compute Wright & Howard F matrix
    
    # Allocate matrix
    F = zeros(Float64, (2*Nplanets+2), length(t))
    
    # Fill matrix
    for i=1:Nplanets
        for j=1:length(t)
            f = f_from_t(P, ecc, t[j], tp)
            F[2*i - 1,j] = cos(f)
            F[2*i, j] = sin(f)
        end 
    end 
    
    # 
    for j=1:length(t)
        F[end-1, j] = 1.0
        F[end, j] = t[j] - t[1]
    end 
    return F
end

WH_F (generic function with 1 method)

In [14]:
F = WH_F(best_period, 0.1, time, 1.0)

4x59 Array{Float64,2}:
  0.96572    0.965716   0.965711   -0.210093  …     0.998194     -0.0971979
 -0.259586  -0.259603  -0.25962    -0.977681       -0.0600714     0.995265 
  1.0        1.0        1.0         1.0             1.0           1.0      
  0.0        0.00068    0.0014    151.344        2554.91       2741.37     

In [15]:
function WH_eps(F::Array{Float64, 2}, W::Array{Float64, 2})
    # Compute Wright & Howard epsilon matrix
    return inv(F * W * (F'))
end 

WH_eps (generic function with 1 method)

In [16]:
WH_eps(F, W)

4x4 Array{Float64,2}:
  0.0857657     0.000335324  -0.0542203    8.1863e-6 
  0.000335324   0.0628552    -0.00516512  -2.22184e-6
 -0.0542203    -0.00516512    0.115636    -5.73326e-5
  8.1863e-6    -2.22184e-6   -5.73326e-5   5.01336e-8

In [17]:
function WH_Beta(RV::Array{Float64, 2}, F::Array{Float64, 2}, W::Array{Float64, 2})
    # Compute Wright & Howard Beta vector
    return RV * W * (F') * WH_eps(F, W)
end 

WH_Beta (generic function with 1 method)

In [18]:
B = WH_Beta(rv', F, W)

1x4 Array{Float64,2}:
 9.83097  297.156  -31.8704  -0.0164657

In [70]:
function rv_forward(P::Float64, ecc::Float64, tp::Float64, t::Array{Float64,1}, rv::Array{Float64,1}, err::Array{Float64,1}; Nplanets::Int=1)
    # Calculates model RV given linear params: [h, c, v0, d, tp]
    
    # Allocate 
    rv_mod = zeros(Float64, Nplanets, length(t))
    
    # Calculate Wright & Howard Beta Vector, B = [h_i, c_i, ..., h_n, c_n, v0, d] for n planets
    B = WH_Beta(rv', WH_F(P, ecc, t, tp), WH_W(err))
    
    # Loop over planets and observations, calculating model rv points
    for i=1:Nplanets
        for j=1:length(t)
            f = f_from_t(P, ecc, t[j], tp)
            rv_mod[i,j] = v_rad_lin(B[i,1], f, B[i,2], B[i,end-1], t[j], t[1], B[i,end])
        end 
    end 
    
    return rv_mod, B
end 

rv_forward (generic function with 1 method)

In [20]:
function loglike(data, model, err)
    # Chi^2
    ll = 0.0;
    for i=1:length(data)
        ll -= 0.5 * (data[i] - model[i])^2 / (err[i])^2;
    end
    return ll
end 

loglike (generic function with 1 method)

In [37]:
ecc = linspace(0.0, 0.99, 1000);
tp = linspace(time[1], time[end], 1000);
period = best_period;

ll_min = -1.0e20;
B_best = nothing;
ecc_best = nothing;
tp_best = nothing;
rv_best = nothing;
for i=1:length(ecc)
    for j=1:length(tp)
        rv_mod, B = rv_forward(period, ecc[i], tp[j], time, rv, err);
        ll = loglike(rv, rv_mod, err);
        if ll > ll_min
            B_best = B;
            ll_min = ll;
            ecc_best = ecc[i];
            tp_best = tp[j];
            rv_best = rv_mod;
        end 
    end 
end         

In [40]:
ecc_best

0.13675675675675675

In [ ]:
function rv_grid(rv_data, time, err; N::Int=1000)    
    ecc = linspace(0.0, 0.99, N);
    tp = linspace(time[1], time[end], N);
    period = best_period;

    ll_min = -1.0e20;
    B_best = nothing;
    ecc_best = nothing;
    tp_best = nothing;
    rv_best = nothing;
    for i=1:length(eccentricity)
        for j=1:length(tp)
            rv_mod, B = rv_forward(period, ecc[i], tp[j], time, rv, err);
            ll = loglike(rv, rv_mod, err);
            if ll > ll_min
                B_best = B;
                ll_min = ll;
                ecc_best = ecc[i];
                tp_best = tp[j];
                rv_best = rv_mod;
            end 
        end 
    end 
    
end 

In [ ]:
ecc_best

In [ ]:
tp_best

In [ ]:
ll_min

In [ ]:
B_best

In [ ]:
plot(time_fold, rv_best', "o", ms=5)
plot(time_fold, rv, "o")
xlabel("Days")
ylabel("RV")

In [21]:
function rv_loglike(rho)
    #rho = [period, ecc, tp]
    
    # hard bounds
    if rho[1] < 0.0
        return Inf
    end 
    if rho[2] >= 1.0
        return Inf
    end 
    if rho[2] < 0.0 
        return Inf
    end
    
    # call forward model 
    model, B = rv_forward(rho[1], rho[2], rho[3], time, rv, err);
    
    # Chi^2
    return -loglike(rv, model, err);
end 

rv_loglike (generic function with 1 method)

In [ ]:
using Optim

In [ ]:
p0 = [best_period, 0.0, mean(time)];
@time optimum = optimize(rv_loglike, p0, autodiff=true)

In [ ]:
model, B = rv_forward(p0[1], p0[2], p0[3], time, rv, err);

In [ ]:
loglike(rv, model, err)

In [ ]:
MLE = optimum.minimum

In [ ]:
function rv_curve_forward(time, p::Vector)
        
    # call forward model 
    model, B = rv_forward(p[1], p[2], p[3], time, rv, err);
        
    return model
    
end

In [ ]:
numbers = readdlm("mystery_planet.txt");

In [ ]:
p0 = [best_period, 0.0, mean(time)];
@time fit3 = curve_fit(rv_curve_forward, time, rv, 1.0./err.^2, p0);

In [68]:
function solve_rv(data::Array{Float64, 2}; p0=[nothing, nothing, nothing])
    # p0 = [period, ecc, tp]
    
    # Unpack RV data (make global?)
    time = data[:,1];
    rv = data[:,2];
    err = data[:,end];
    
    # Set initial parameters if not specified
    p = [0.0, 0.0, 0.0]
    if p0[1] == nothing
        # Use Agol Periodogram for initial period guess
        periods = collect(linspace(0.1, time[end]-time[1], 10000))
        p[1] = agol_periodogram(numbers, periods)
    else
        p[1] = p0[1]
    end 
    if p0[2] == nothing
        # Use random ecc [0,1) for initial guess
        p[2] = rand()
    else
        p[2] = p0[2]
    end 
    if p0[3] == nothing
        # Use random time drawn from observed grid
        p[3] = rand(time)
    else
        p[3] = p0[3]
    end
    
    # Use optimize to solve 
    optimum = optimize(rv_loglike, p, autodiff=true)
    
    return optimum.minimum
    
end 

solve_rv (generic function with 1 method)

In [71]:
pbest = solve_rv(numbers, p0=[nothing, 0.4, 13500.])

3-element Array{Float64,1}:
   116.702   
     0.129161
 13541.2     